In [ ]:
import pandas as pd
import os
import sys
import numpy as np
from matplotlib import pyplot as plt

sys.path.append("..")
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import rbf_functions

rbfs = [
    rbf_functions.original_rbf,
    rbf_functions.squared_exponential_rbf,
    rbf_functions.inverse_quadratic_rbf,
    rbf_functions.inverse_multiquadric_rbf,
    rbf_functions.exponential_rbf,
    rbf_functions.matern32_rbf,
    rbf_functions.matern52_rbf,
]

def format_rbf_name(name):
    # Convert underscores to spaces and capitalize the first letter of each word
    formatted_name = ' '.join(word.capitalize() for word in name.split('_'))

    # Convert "Rbf" to "RBF"
    formatted_name = formatted_name.replace("Rbf", "")
    formatted_name = formatted_name.replace("Original", "Modified SE")

    # Convert "Matern32" to "Matern 32" and "Matern52" to "Matern 52"
    formatted_name = formatted_name.replace("Matern32", "Matern 32")
    formatted_name = formatted_name.replace("Matern52", "Matern 52")

    return formatted_name

In [ ]:
sns.set_style("white")

In [ ]:
global_hv = pd.read_csv("./hv/hypervolume_global_refset.csv", index_col="Unnamed: 0")
global_hv = global_hv.iloc[0, 0]

In [ ]:
root_dir = "../output"
fig, axes = plt.subplots(5, 7, sharex=True, sharey="row", figsize=(20, 14))  #
fontsize = 18
plt.rcParams.update({
    'font.size': fontsize,               # Default fontsize for text added to plots (xlabel, ylabel, title, etc.)
    'axes.labelsize': fontsize,          # Fontsize of the x and y labels
    'xtick.labelsize': fontsize,         # Fontsize of the tick labels
    'ytick.labelsize': fontsize,         # Fontsize of the tick labels
    'legend.fontsize': fontsize,         # Fontsize in the legend
    'axes.titlesize': 20          # Fontsize of subplot titles
})
lw = 1.5

for i, (entry, c) in enumerate(zip(rbfs, sns.color_palette())):
    rbf = entry.__name__
    df_hv = pd.read_csv(f"./hv/{rbf}_hv_global.csv")
    df_hv /= global_hv

    df_hv_nfe = pd.read_csv(f"./hv/{rbf}_nfe.csv")
    df_ei = pd.read_csv(f"./epind/{rbf}_ei_all.csv")
    df_ei_nfe = pd.read_csv(f"./epind/{rbf}_ei_nfe_all.csv")
    df_gd = pd.read_csv(f"./gd/{rbf}_gd_all.csv")
    df_gd_nfe = pd.read_csv(f"./gd/{rbf}_gd_nfe_all.csv")
    ep_nfe = {}
    ep_prog = {}
    archive_nfe = {}
    archive_count = {}
    out_dir = os.path.join(root_dir, rbf)

    for j in os.listdir(out_dir):
        if j.endswith("_convergence.csv"):
            ep_load = pd.read_csv(os.path.join(out_dir, j))
            ep_nfe[int(j.split("_")[0])] = ep_load.nfe
            ep_prog[int(j.split("_")[0])] = ep_load.improvements
        if j.endswith("_hypervolume.csv"):
            archives_by_nfe = pd.read_csv(os.path.join(out_dir, j))
            dups = archives_by_nfe.pivot_table(columns=["Unnamed: 0"], aggfunc="size")
            archive_nfe[int(j.split("_")[0])] = dups.index
            archive_count[int(j.split("_")[0])] = dups.reset_index(drop=True)
    ep_nfe = pd.DataFrame.from_dict(ep_nfe)
    ep_prog = pd.DataFrame.from_dict(ep_prog)
    archive_nfe = pd.DataFrame.from_dict(archive_nfe, orient="index").T
    archive_size = pd.DataFrame.from_dict(archive_count, orient="index").T

    for j in range(len(df_gd_nfe.columns)):
        axes[0, i].plot(df_gd_nfe.iloc[:, j], df_gd.iloc[:, j], color=c, lw = lw)
        axes[1, i].plot(df_ei_nfe.iloc[:, j], df_ei.iloc[:, j], color=c, lw = lw)
        axes[2, i].plot(df_hv_nfe.iloc[:, j], df_hv.iloc[:, j], color=c, lw = lw)

        axes[3, i].plot(ep_nfe.iloc[:, j], ep_prog.iloc[:, j], color=c, lw = lw)
        axes[4, i].plot(
            archive_nfe.iloc[:, j], archive_size.iloc[:, j], color=c, lw = lw
        )

    for ax in axes[:, i]:
        ax.set_box_aspect(1)

    if i == 0:
        axes[0, i].set_ylabel("Generational Distance", labelpad=12)
        axes[1, i].set_ylabel("Epsilon Indicator", labelpad=12)
        axes[2, i].set_ylabel("Hypervolume", labelpad=12)  # , rotation=45, labelpad=40
        axes[3, i].set_ylabel("Epsilon Progress", labelpad=4)
        axes[4, i].set_ylabel("Archive Size", labelpad=4)

    # axes[4, i].set_xlabel("NFE")
    axes[0, i].set_title(format_rbf_name(rbf[:-4]), y= 1.09)


for row in axes[0:3]:
    ax = row[0]
    ax.set_ylim(ymin=0, ymax=1)


fig.tight_layout()
fig.text(0.5, -0.0005, 'Number of function evaluations', ha='center', va='center', fontsize=fontsize)
plt.savefig("../figs/paper_figures/png/metrics_LakeProblem.png", bbox_inches='tight')
plt.savefig("../figs/paper_figures/pdf/metrics_LakeProblem.pdf", bbox_inches='tight')